In [1484]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/popopo/cheat.csv
/kaggle/input/ai-camp-3/test_imdb.csv
/kaggle/input/ai-camp-3/train_imdb.csv
/kaggle/input/ai-camp-3/sbm file.csv


In [1485]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from xgboost import XGBRegressor
pd.set_option('display.max_columns',None)
numeric_imputer = SimpleImputer(strategy='mean')
from sklearn.preprocessing import StandardScaler

In [1486]:
test=pd.read_csv("/kaggle/input/ai-camp-3/test_imdb.csv")
df=pd.read_csv("/kaggle/input/ai-camp-3/train_imdb.csv")
sub=pd.read_csv("/kaggle/input/ai-camp-3/sbm file.csv")


In [ ]:
from ydata_profiling import ProfileReport
profiletrain = ProfileReport(df, explorative=True)
profiletest = ProfileReport(test, explorative=True)



In [1487]:
top_countrie=['KR']

In [1488]:

# Function to assign country group
def assign_country_group(countries):
    # Ensure that countries is a list or string
    if isinstance(countries, list):
        for country in countries:
            if country.strip().lower() in [c.strip().lower() for c in top_countrie]:
                return country
    elif isinstance(countries, str):
        # In case there's a string instead of a list (handle for other cases)
        if countries.strip().lower() in [c.strip().lower() for c in top_countrie]:
            return countries
    return 'Others'

# Apply the function to both train and test data
df['country_group'] = df['production_countries'].apply(assign_country_group)
test['country_group'] = test['production_countries'].apply(assign_country_group)

# Verify the result
print(df[['production_countries', 'country_group']].head())

dy = pd.get_dummies(df, columns=['country_group',"type"], drop_first=False)
tes = pd.get_dummies(test, columns=['country_group',"type"], drop_first=False)

  production_countries country_group
0                   FR        Others
1                   GB        Others
2                   US        Others
3                   US        Others
4                   IN        Others


In [1489]:
dg = pd.get_dummies(df, columns=['genres'], drop_first=False)
tesg = pd.get_dummies(test, columns=['genres'], drop_first=False)

In [1490]:



# Preprocessing: Selecting only numerical columns
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns

# Ensure the target variable is included in the numerical columns
target = 'imdb_score'
if target not in numerical_columns:
    numerical_columns = numerical_columns.insert(0, target)


# Filter the train and test datasets
train_numerical = df[numerical_columns]
test_numerical = test[numerical_columns.drop(target, errors='ignore')]

# Fill numerical columns (except 'seasons') with their mean
train_numerical = train_numerical.copy()
test_numerical = test_numerical.copy()

train_numerical.loc[:, train_numerical.columns != 'seasons'] = \
    train_numerical.loc[:, train_numerical.columns != 'seasons'].fillna(train_numerical.mean())

test_numerical.loc[:, test_numerical.columns != 'seasons'] = \
    test_numerical.loc[:, test_numerical.columns != 'seasons'].fillna(train_numerical.mean())

# Fill 'seasons' column with 0
train_numerical['seasons'] = train_numerical['seasons'].fillna(0)
test_numerical['seasons'] = test_numerical['seasons'].fillna(0)



# Splitting train data into training and validation sets
X = train_numerical.drop(columns=[target])
y = train_numerical[target]



In [1491]:
oo=dg.iloc[:, -19:]
oot=tesg.iloc[:, -19:]
#ooo=oo[["genres_drama","genres_comedy","genres_documentation"]]
#ooot=oot[["genres_drama","genres_comedy","genres_documentation"]]
X['type_SHOW']=dy['type_SHOW']
X['country_group_KR']=dy['country_group_KR']
X['country_group_Others']=dy['country_group_Others']

test_numerical['country_group_KR']=tes['country_group_KR']
test_numerical['country_group_Others']=tes['country_group_Others']
test_numerical['type_SHOW']=tes['type_SHOW']

X1 = pd.concat([X, oo], axis=1)
test_numerical1 = pd.concat([test_numerical, oot], axis=1)
#X2 = X1.drop(columns=['genres_western'])
#test_numerical2 = test_numerical1.drop(columns=['genres_western'])


In [1492]:
X4=X1.copy()
test_numerical4=test_numerical1.copy()
X4['seasons'] = X4['seasons'].apply(lambda x: 'film' if x == 0 else ('long' if x > 3 else 'short'))
test_numerical4['seasons'] = test_numerical4['seasons'].apply(lambda x: 'film' if x == 0 else ('long' if x > 3 else 'short'))
X4 = pd.get_dummies(X4, columns=['seasons'], drop_first=False)
test_numerical4 = pd.get_dummies(test_numerical4, columns=['seasons'], drop_first=False)

In [1493]:
X4 = X4.drop(columns=['seasons_film'])
test_numerical4 = test_numerical4.drop(columns=['seasons_film'])

In [1494]:
#X5 = X5.drop(columns=['release_year_old'])
#test_numerical5 = test_numerical5.drop(columns=['release_year_old'])

In [1495]:
#X6=X4.copy()
#test_numerical6=test_numerical4.copy()
#X6['release_year'] = X6['release_year'].apply(lambda x: 'new' if x > 2005 else 'old')
#test_numerical6['release_year'] = test_numerical6['release_year'].apply(lambda x: 'new' if x > 2005 else 'old')
#X6 = pd.get_dummies(X6, columns=['release_year'], drop_first=False)
#test_numerical6 = pd.get_dummies(test_numerical6, columns=['release_year'], drop_first=False)

In [1496]:
#X4 = X3.drop(columns=['release_year'])
#test_numerical4 = test_numerical3.drop(columns=['release_year'])

In [1497]:
#from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
#scaler = MinMaxScaler()

# Scale the 'release_year' column
#X4['imdb_votes'] = scaler.fit_transform(X4[['imdb_votes']])
#test_numerical4['imdb_votes'] = scaler.fit_transform(test_numerical4[['imdb_votes']])


In [1498]:

model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
model.fit(X4.drop(columns=['country_group_Others']), y)

# Predict on validation se

# Predict on the test set
test_predictions = model.predict(test_numerical4.drop(columns=['country_group_Others']))

# Prepare the submission file
sub['imdb_score'] = test_predictions
sub.to_csv('submission42.csv', index=False)

print("Submission file created as 'submission.csv'")

Submission file created as 'submission.csv'


In [1499]:
X4

,release_year,runtime,imdb_votes,tmdb_popularity,tmdb_score,type_SHOW,country_group_KR,country_group_Others,country_group_BG,genres_Unknown,genres_action,genres_animation,genres_comedy,genres_crime,genres_documentation,genres_drama,genres_family,genres_fantasy,genres_history,genres_horror,genres_music,genres_reality,genres_romance,genres_scifi,genres_sport,genres_thriller,genres_war,genres_western,seasons_long,seasons_short
0,2022,119,1768.0,255.762,5.6,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,2019,52,251168.0,309.755,8.4,True,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False
2,2020,43,291.0,6.128,6.8,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True
3,2020,49,2759.0,10.962,7.0,True,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
4,2019,128,859.0,1.632,5.0,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4639,2020,98,1195.0,8.844,5.9,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4640,2021,94,3000.0,22.899,7.1,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4641,2021,109,8223.0,40.022,6.9,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
4642,2022,104,368.0,175.354,5.6,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False


In [1500]:
X4.describe()

,release_year,runtime,imdb_votes,tmdb_popularity,tmdb_score
count,4644.000000,4644.000000,4.644000e+03,4644.000000,4644.000000
mean,2016.080965,77.861542,2.384330e+04,22.864471,6.818025
std,7.241289,39.531229,8.714775e+04,70.602236,1.145470
min,1945.000000,0.000000,5.000000e+00,0.600000,1.000000
25%,2016.000000,44.750000,6.025000e+02,3.156000,6.175000
50%,2018.000000,84.000000,2.898500e+03,7.645000,6.818025
75%,2020.000000,105.000000,1.934050e+04,18.978500,7.500000
max,2022.000000,251.000000,2.268288e+06,1823.374000,10.000000
